# Simple Time Series Example

A first impression for the execution of a time series simulation has already been given by the [example](https://pandapipes.readthedocs.io/en/latest/timeseries/example.html) in the documentation. Here a new pandapipes network was created. In the following, an already existing net is loaded from a json file. The prerequisites for a successful execution of a time series simulation are:

- a pandapipes network
- predefined time series for occurring sinks and sources in the network (e.g. 2 csv files)

First, the following imports must be carried out for a general timer series simulation:

In [ ]:
import os
import pandas as pd
import pandapower.control as control
import pandapipes
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapipes.timeseries import run_timeseries_ppipe

Since in this example an already existing network is loaded, the upcoming code must be added:

In [ ]:
from pandapipes import networks

Now the pandapipes network can be created/ loaded:

In [ ]:
net = pandapipes.networks.simple_gas_networks.gas_meshed_delta()

This is the delta STANET network from the chapter [Meshed Networks](https://pandapipes.readthedocs.io/en/latest/networks/meshed/meshed_networks.html) with the medium gas.

<img src="pics/time_series_simple_example_network.png">

Furthermore, the network can be prepared. For this purpose the time series for sinks and sources have to be defined. Since there is only one sink and one source, the respective csv file only consists of one column. The number of lines corresponds to the desired number of time steps, in this case 10.

<img src="pics/time_series_simple_example_csv_files.png">

With the code

In [ ]:
profiles_sink = pd.read_csv(os.path.join(os.path.dirname(__file__), 'files',
                                         'simple_time_series_example_sink_profiles.csv'),
                                         index_col=0)
profiles_source = pd.read_csv(os.path.join(os.path.dirname(__file__), 'files',
                                           'simple_time_series_example_source_profiles.csv'),
                                           index_col=0)
ds_sink = DFData(profiles_sink)
ds_source = DFData(profiles_source)

const_sink = control.ConstControl(net, element='sink', variable='mdot_kg_per_s',
                                  element_index=net.sink.index.values, data_source=ds_sink,
                                  profile_name=net.sink.index.values.astype(str))
const_source = control.ConstControl(net, element='source', variable='mdot_kg_per_s',
                                    element_index=net.source.index.values,
                                    data_source=ds_source,
                                    profile_name=net.source.index.values.astype(str))
del const_sink.initial_powerflow
const_sink.initial_pipeflow = False
del const_source.initial_powerflow
const_source.initial_pipeflow = False

the corresponding csv-file is read out and the resulting DataFrames are written into the network with the help of the controller [ConstControl](https://pandapower.readthedocs.io/en/v2.2.2/control/controller.html#constcontrol). Then the desired time steps for which the network is to be calculated must be specified. In the present case, these can be defined from 1 to 10 at most, since only 10 time steps are specified for the sink and source. Likewise, for example, a range of 3 to 6 can be specified. Here the time steps from 1 to 10 are defined with

In [ ]:
time_steps = range(10)

Furthermore an [OuputWriter](https://pandapower.readthedocs.io/en/v2.2.2/timeseries/output_writer.html) must be created, which is defined as `ow` using

In [ ]:
log_variables = [('res_junction', 'p_bar'), ('res_pipe', 'v_mean_m_per_s'),
                 ('res_pipe', 'reynolds'), ('res_pipe', 'lambda'),
                 ('res_sink', 'mdot_kg_per_s'), ('res_source', 'mdot_kg_per_s'),
                 ('res_ext_grid', 'mdot_kg_per_s')]
ow = OutputWriter(net, time_steps, output_path=None, log_variables=log_variables)

In this the results of the simulation are stored. Finally, the main function for starting the simulation can be called:

In [ ]:
run_timeseries_ppipe(net, time_steps, output_writer=ow)

The complete programme is shown below:

In [ ]:
import os
import pandas as pd
import pandapower.control as control
import pandapipes
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapipes.timeseries import run_timeseries_ppipe
from pandapipes import networks

# 1. create/ load the pandapipes network
net = pandapipes.networks.simple_gas_networks.gas_meshed_delta()

# 2. preparing the network for simulation
profiles_sink = pd.read_csv(os.path.join(os.path.dirname(__file__), 'files',
                                         'simple_time_series_example_sink_profiles.csv'),
                                         index_col=0)
profiles_source = pd.read_csv(os.path.join(os.path.dirname(__file__), 'files',
                                           'simple_time_series_example_source_profiles.csv'),
                                           index_col=0)
ds_sink = DFData(profiles_sink)
ds_source = DFData(profiles_source)

const_sink = control.ConstControl(net, element='sink', variable='mdot_kg_per_s',
                                  element_index=net.sink.index.values, data_source=ds_sink,
                                  profile_name=net.sink.index.values.astype(str))
const_source = control.ConstControl(net, element='source', variable='mdot_kg_per_s',
                                    element_index=net.source.index.values,
                                    data_source=ds_source,
                                    profile_name=net.source.index.values.astype(str))
del const_sink.initial_powerflow
const_sink.initial_pipeflow = False
del const_source.initial_powerflow
const_source.initial_pipeflow = False

# 3. define the time steps to be calculated
time_steps = range(10)

# 4. creating an output writer
log_variables = [('res_junction', 'p_bar'), ('res_pipe', 'v_mean_m_per_s'),
                 ('res_pipe', 'reynolds'), ('res_pipe', 'lambda'),
                 ('res_sink', 'mdot_kg_per_s'), ('res_source', 'mdot_kg_per_s'),
                 ('res_ext_grid', 'mdot_kg_per_s')]
ow = OutputWriter(net, time_steps, output_path=None, log_variables=log_variables)

# 5. run simulation
run_timeseries_ppipe(net, time_steps, output_writer=ow)

The results of the simulation can be accessed via the OutputWriter as follows:

In [ ]:
print("pressure:")
print(ow.np_results["res_junction.p_bar"])
print("mean velocity:")
print(ow.np_results["res_pipe.v_mean_m_per_s"])
print("reynolds number:")
print(ow.np_results["res_pipe.reynolds"])
print("lambda:")
print(ow.np_results["res_pipe.lambda"])
print("mass flow sink:")
print(ow.np_results["res_sink.mdot_kg_per_s"])
print("mass flow source:")
print(ow.np_results["res_source.mdot_kg_per_s"])
print("mass flow ext. grid:")
print(ow.np_results["res_ext_grid.mdot_kg_per_s"])

To view the results more clearly, they can be displayed with `matplotlib.pyplot` in a diagram with the time steps as x-axis.The time curve for the mass flow in the External Grid looks as follows:

<img src="pics/time_series_simple_example_plot_mass_flow_ext_grid.png">

To do this, the code

In [ ]:
import matplotlib.pyplot as plt

x = time_steps
y = ow.np_results["res_ext_grid.mdot_kg_per_s"]

plt.xlabel("time step")
plt.ylabel("mass flow [kg/s]")
plt.title("External Grid Mass Flow Profile")
plt.plot(x, y, "r-o")
plt.grid()
plt.show()

must be executed. In addition, the pressure curves for the individual junctions can be displayed with

In [ ]:
import matplotlib.pyplot as plt

x = time_steps
y = ow.np_results["res_junction.p_bar"]

plt.xlabel("time step")
plt.ylabel("pressure [bar]")
plt.title("Pressure Profile")
plt.plot(x, y[:,0], "g-o")
plt.plot(x, y[:,1], "r-o")
plt.plot(x, y[:,2], "y-o")
plt.legend(["Source", "External Grid", "Sink"], loc='upper left')
plt.grid()
plt.show()

in a graph:

<img src="pics/time_series_simple_example_plot_pressures.png">

As a final example to illustrate the results, the flow velocities of the three pipes are plotted:

<img src="pics/time_series_simple_example_plot_velocities.png">

For this purpose, the following code must be executed:

In [ ]:
import matplotlib.pyplot as plt

x = time_steps
y = ow.np_results["res_pipe.v_mean_m_per_s"]

plt.xlabel("time step")
plt.ylabel("v_mean [m/s]")
plt.title("Velocity Profile")
plt.plot(x, y[:,0], "g-o")
plt.plot(x, y[:,1], "r-o")
plt.plot(x, y[:,2], "y-o")
plt.legend(["Pipe 1", "Pipe 2", "Pipe 3"], loc='lower left')
plt.grid()
plt.show()